In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 2.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from safetensors.torch import load_file
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

In [3]:
file_path = '/kaggle/input/output1/output1.csv'
df1 = pd.read_csv(file_path)

file_path_2 = '/kaggle/input/law-data-merge/law_data_2.csv'
df2 = pd.read_csv(file_path_2)
df = pd.concat([df1, df2], ignore_index=True)

In [4]:
df['전문'] = df['전문'].fillna('')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

In [7]:
okt = Okt()

In [8]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

In [9]:
def chunk_text(text, chunk_size=128, overlap=32):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

In [10]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

In [11]:
df['processed_text'] = df['전문'].apply(preprocess_text)
df['chunks'] = df['processed_text'].apply(lambda x: chunk_text(x))

In [12]:
chunk_embeddings = []
chunk_to_doc = []

In [13]:
for idx, chunks in enumerate(df['chunks']):
    for chunk in chunks:
        chunk_embeddings.append(encode_text(chunk))
        chunk_to_doc.append(idx)

In [14]:
chunk_embeddings = torch.stack(chunk_embeddings).to(device)

In [15]:
def search_query(query, df, chunk_embeddings, chunk_to_doc, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0).to(device)

    query_embedding_cpu = query_embedding.cpu()
    chunk_embeddings_cpu = chunk_embeddings.cpu()

    similarities = cosine_similarity(query_embedding_cpu, chunk_embeddings_cpu).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    results = df.iloc[[chunk_to_doc[i] for i in top_indices]]
    return results[['주문']]


In [16]:
#query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
query = '고양시 일산동구 B에 있는 C 1층에 있는 피해자 D가 운영하는 E에서, 계산대에 올려 놓은 우유 2팩을 바닥으로 던져 우유가 쏟아지게 하고, 소리를 지르는 등 소란을 피워 위력으로써 피해자의 제과점 운영 업무를 방해하였다'
result = search_query(query, df, chunk_embeddings, chunk_to_doc)

In [17]:
print(result)

                                                     주문
654   피고인을 벌금 500,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우...
1047                               피고인을 징역 1년 2개월에 처한다.
1033  피고인을 징역 1년에 처한다. 압수된 식칼 1개(울산지방검찰청 2019년 압 제32...
1129  피고인을 징역 1년에 처한다. 압수된 식칼 1개(울산지방검찰청 2019년 압 제32...
1675                               피고인을 징역 1년 8개월에 처한다.


In [18]:
pd.set_option('display.max_colwidth', None)
row_index = 1033
print(df.at[row_index, '전문'])

범죄사실 및 치료감호원인사실 [범죄전력] 피고인 겸 피치료감호청구인(이하, ‘피고인’이라 한다)은 2019. 2. 28. 울산지방법원에서  공무집행방해죄 등으로 징역 8월, 집행유예 2년을 선고받아 현재 항소심 계속 중이다. [범죄사실] 피고인은 상세불명의 양극성 정동장애, 알코올 사용의 의존증후군 등으로 인하여 사물을 변별하거나 의사를 결정할 능력이 미약한 상태에서 아래와 같은 범행을 하였다. 『2019고합167』 1. 사기 가. 피고인은 2019. 3. 17. 01:04경 울산 북구 &&동 ##차고지에서 사실은 택시요금을 지불할 의사나 능력이 없음에도, 마치 요금을 지불할 것처럼 행세하며 피해자 C(63세)가 운행하는 울산 D호 택시에 승차하여 이에 속은 피해자로 하여금 울산 중구 **로  00 ○○지구대 앞 노상까지 운행하게 하여 택시요금 11,800원 상당의 재산상 이익을  취득하였다. 나. 피고인은 2019. 3. 19. 01:05경 울산 남구 □□박물관 앞에서 사실은 택시요금을  지불할 의사나 능력이 없음에도, 마치 요금을 지불할 것처럼 행세하며 피해자 E(59세)가 운행하는 울산 F호 택시에 승차하여 이에 속은 피해자로 하여금 울산 중구 **로  00 ○○지구대 앞 노상까지 운행하게 하여 택시요금 15,600원1)상당의 재산상 이익을  취득하였다. 다. 피고인은 2019. 4. 21. 23:00경 울산 남구 **로 00 피해자 G(여, 50세)가 운영하는 '△△' 주점 내에서 손님으로 들어가 사실은 술값을 지불할 의사나 능력이 없음에도, 마치 술값을 지불할 것처럼 행세하며 주류를 주문하여 이에 속은 피해자로부터 맥주 5병, 안주 등 합계 4만 원 상당의 술과 안주를 제공 받았다. 이로써 피고인은 피해자들을 기망하여 재물을 교부받거나 재산상 이익을 취득하였다. 2. 공무집행방해 피고인은 2019. 4. 26. 09:30경 울산 남구 **로 00 ‘♧♧주민센터’에 전입신고를 하러 가 혼잣말로 욕설을 하는 등 이상한 행동을 하여 복지담당 직원 H가 방문

In [19]:
###################################################################################################################################################################

In [20]:
pip install transformers safetensors scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip install konlpy

In [22]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import RobertaTokenizer, RobertaModel
from safetensors.torch import load_file
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import os

In [23]:
file_path = '/kaggle/input/output1/output1.csv'
df = pd.read_csv(file_path)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

In [26]:
okt = Okt()

In [27]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

In [28]:
df['전문'] = df['전문'].apply(preprocess_text)

In [29]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

In [30]:
def search_query(query, df, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0)

    document_embeddings = [encode_text(text) for text in df['전문']]
    document_embeddings = torch.stack(document_embeddings)

    similarities = cosine_similarity(query_embedding.cpu(), document_embeddings.cpu()).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    results = df.iloc[top_indices]
    return results[['주문']]

In [31]:
query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
result = search_query(query, df)

In [32]:
print(result)

                                                                                                주문
637        피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 3년간 위 형의 집행을 유예한다. 피고인에게 보호관찰을 받을 것과 80시간의 사회봉사를 각 명한다.
944                     피고인을 징역 8개월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 보호관찰을 받을 것을 명한다.
948                     피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 3년간 위 형의 집행을 유예한다. 압수된 과도 1개(증 제1호)를 몰수한다.
771                                             피고인을 징역 8월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다.
83   피고인을 징역 6월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 80시간의 사회봉사 및 40시간의 알코올 치료강의 수강을 명한다.


In [33]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from safetensors.torch import load_file
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch

file_path = '/kaggle/input/output1/output1.csv'
df = pd.read_csv(file_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

okt = Okt()

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

df['전문'] = df['전문'].apply(preprocess_text)

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

def search_query(query, df, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0)

    chunk_size = 100
    document_chunks = [df['전문'].iloc[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
    document_embeddings = [encode_text(chunk.str.cat(sep=' ')) for chunk in document_chunks]
    document_embeddings = torch.stack(document_embeddings)

    similarities = cosine_similarity(query_embedding.cpu(), document_embeddings.cpu()).flatten()

    top_chunk_indices = similarities.argsort()[-top_k:][::-1]

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([df['전문'].iloc[i*chunk_size:(i+1)*chunk_size].str.cat(sep=' ') for i in top_chunk_indices])
    query_tfidf = tfidf_vectorizer.transform([query])
    tfidf_scores = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    final_scores = []
    for i in range(len(similarities)):
        if i < len(tfidf_scores):
            final_scores.append(similarities[i] * tfidf_scores[i])
        else:
            final_scores.append(similarities[i])

    top_indices = sorted(range(len(final_scores)), key=lambda i: final_scores[i], reverse=True)[:top_k]

    results = df.iloc[top_indices]
    return results[['주문']]

query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
result = search_query(query, df)
print(result)


                                                                                                                                                                                                            주문
6                                                                                                                                                                                 원심판결을 파기한다. 피고인을 징역 1년에 처한다.
9  피고인을 벌금 500만 원에 처한다. 다만 이 판결 확정일로부터 1년간 위 형의 집행을 유예한다. 위 집행유예의 선고가 실효 또는 취소되고 피고인이 위 벌금을 납입하지 아니하는 경우 10만 원을 1일로 환산한 기간 피고인을 노역장에 유치한다. 이 사건 공소사실 중 근로자 B, C에 대한 임금 등 미지급을 이유로 한 근로기준법위반의 점에 대한 공소를 각 기각한다.
5                                                                                                        피고인을 벌금 300만 원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 10만 원을 1일로 환산한 기간 피고인을 노역장에 유치한다. 위 벌금에 상당한 금액의 가납을 명한다.
8                        피고인 A를 벌금 300만 원에, 피고인 B을 징역 8개월에 각 처한다. 피고인 A가 위 벌금을 납입하지 아니하는 경우 10만 원을 1일로 환산한 기간 위 피고인을 노역장에 유치한다. 다만, 이 판결 확정일로부터 2년간 피고인 B에 대한 위 형의 집행을 유예한다.

In [34]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from safetensors.torch import load_file
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

file_path = '/kaggle/input/output1/output1.csv'
df = pd.read_csv(file_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

okt = Okt()
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

def chunk_text(text, chunk_size=128, overlap=32):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

df['processed_text'] = df['전문'].apply(preprocess_text)
df['chunks'] = df['processed_text'].apply(lambda x: chunk_text(x))

chunk_embeddings = []
chunk_to_doc = []

for idx, chunks in enumerate(df['chunks']):
    for chunk in chunks:
        chunk_embeddings.append(encode_text(chunk).cpu())
        chunk_to_doc.append(idx)

chunk_embeddings = torch.stack(chunk_embeddings)

def search_query(query, df, chunk_embeddings, chunk_to_doc, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0).cpu()

    similarities = cosine_similarity(query_embedding, chunk_embeddings).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    results = df.iloc[[chunk_to_doc[i] for i in top_indices]]
    return results[['주문']]

# Query example
query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
result = search_query(query, df, chunk_embeddings, chunk_to_doc)

print(result)


                                                                                                                                          주문
948                                                               피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 3년간 위 형의 집행을 유예한다. 압수된 과도 1개(증 제1호)를 몰수한다.
607  피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 보호관찰을 받을 것과 80시간의 사회봉사 및 40시간의 폭력치료강의 수강을 명한다. 압수된 과도 1점(증 제1호)을 피고인으로부터 몰수한다.
510                                                                                       피고인을 징역 6월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다.
956                                        피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 40시간의 알코올 치료강의 수강을 명한다. 압수된 증 제2호를 몰수한다.
253                                                                                                 피고인을 징역 6년에 처한다. 압수된 식칼 1개(증 제1호)를 몰수한다.


In [35]:
pd.set_option('display.max_colwidth', None)
row_index = 948
print(df.at[row_index, '전문'])

범 죄 사 실 피고인은 2023. 3. 19. 08:30경 시흥시 B, C호 피고인의 주거지   내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다. 이로써 피고인은 위험한 물건을 휴대하여 피해자에게 상해를 가하였다. 증거의 요지 1. 피고인의 법정진술 1. D에 대한 진술조서 1. 입건전조사보고서(현장상황 및 피해자 진술), 각 수사보고서(편의점  CCTV 확인 /    방범용 CCTV 확인 / 관련 112신고사건처리표 첨부) 1. 압수조서, 현장 사진 및 압수물 사진 1. 각 수사보고서(피해자 제출 상해 진단서 / 소방 구급활동일지 첨부), 수사보고(피해자 전화진술 청취) 법령의 적용 1. 범죄사실에 대한 해당법조 형법 제258조의2 제1항, 제257조 제1항 1. 집행유예 형법 제62조 제1항 1. 몰수 형법 제48조 제1항 제1호
